# Машинное обучение, ФКН ВШЭ

## Практическое задание 7. Градиентный бустинг ~~своими руками~~

### Общая информация
Дата выдачи: 07.12.2018

Мягкий дедлайн: 05:59MSK 18.12.2018

Жесткий дедлайн: 05:59MSK 19.12.2018

### Оценивание и штрафы
Каждая из задач имеет определенную «стоимость» (указана в скобках около задачи). Максимально допустимая оценка за работу — 10 баллов.

Сдавать задание после указанного срока сдачи нельзя. При выставлении неполного балла за задание в связи с наличием ошибок на усмотрение проверяющего предусмотрена возможность исправить работу на указанных в ответном письме условиях.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов (подробнее о плагиате см. на странице курса). Если вы нашли решение какого-то из заданий (или его часть) в открытом источнике, необходимо указать ссылку на этот источник в отдельном блоке в конце вашей работы (скорее всего вы будете не единственным, кто это нашел, поэтому чтобы исключить подозрение в плагиате, необходима ссылка на источник).

Неэффективная реализация кода может негативно отразиться на оценке.

### Формат сдачи
Задания сдаются через систему anytask. Посылка должна содержать:
* Ноутбук homework-practice-07-Username.ipynb

Username — ваша фамилия и имя на латинице именно в таком порядке

__Задание 1. (0.5 балла)__

Мы будем использовать данные соревнования [Home Credit Default Risk](https://www.kaggle.com/c/home-credit-default-risk/data). 
* Загрузите таблицу application_train.csv;
* Запишите в Y столбец с целевой переменной;
* Удалите ненужные столбцы (для этого воспользуйтесь описанием);
* Определите тип столбцов и заполните пропуски - стратегия произвольная;
* Разбейте выборку в соотношении 70:30 с random_state=0.

Так как в данных значительный дисбаланс классов, в качестве метрики качества везде будем использовать площадь под precision-recall кривой.

In [57]:
import pandas as pd
import numpy as np

In [58]:
df = pd.read_csv('data/application_train.csv')
df = df[:1000]
y_data = df['TARGET']
x_data = df.drop(['TARGET'], axis=1)

In [59]:
np.unique(y_data.values)

array([0, 1])

In [60]:
x_data.head()
print(x_data.columns.values)

['SK_ID_CURR' 'NAME_CONTRACT_TYPE' 'CODE_GENDER' 'FLAG_OWN_CAR'
 'FLAG_OWN_REALTY' 'CNT_CHILDREN' 'AMT_INCOME_TOTAL' 'AMT_CREDIT'
 'AMT_ANNUITY' 'AMT_GOODS_PRICE' 'NAME_TYPE_SUITE' 'NAME_INCOME_TYPE'
 'NAME_EDUCATION_TYPE' 'NAME_FAMILY_STATUS' 'NAME_HOUSING_TYPE'
 'REGION_POPULATION_RELATIVE' 'DAYS_BIRTH' 'DAYS_EMPLOYED'
 'DAYS_REGISTRATION' 'DAYS_ID_PUBLISH' 'OWN_CAR_AGE' 'FLAG_MOBIL'
 'FLAG_EMP_PHONE' 'FLAG_WORK_PHONE' 'FLAG_CONT_MOBILE' 'FLAG_PHONE'
 'FLAG_EMAIL' 'OCCUPATION_TYPE' 'CNT_FAM_MEMBERS' 'REGION_RATING_CLIENT'
 'REGION_RATING_CLIENT_W_CITY' 'WEEKDAY_APPR_PROCESS_START'
 'HOUR_APPR_PROCESS_START' 'REG_REGION_NOT_LIVE_REGION'
 'REG_REGION_NOT_WORK_REGION' 'LIVE_REGION_NOT_WORK_REGION'
 'REG_CITY_NOT_LIVE_CITY' 'REG_CITY_NOT_WORK_CITY'
 'LIVE_CITY_NOT_WORK_CITY' 'ORGANIZATION_TYPE' 'EXT_SOURCE_1'
 'EXT_SOURCE_2' 'EXT_SOURCE_3' 'APARTMENTS_AVG' 'BASEMENTAREA_AVG'
 'YEARS_BEGINEXPLUATATION_AVG' 'YEARS_BUILD_AVG' 'COMMONAREA_AVG'
 'ELEVATORS_AVG' 'ENTRANCES_AVG' 'FLOORSMAX_AVG'

In [61]:
eps = 1e-2
numeric = [np.integer, np.float]
numeric_features = []
for col in x_data.columns.values:
    if x_data[col].values.dtype in numeric:
        if x_data[col].values.var() < eps:
            x_data = x_data.drop(col, axis=1)
            print('Dropped ' + col)
            continue
        numeric_features.append(col)
        x_data[col] = np.nan_to_num(x_data[col].values)
x_data.head()

Dropped REGION_POPULATION_RELATIVE
Dropped FLAG_MOBIL
Dropped FLAG_CONT_MOBILE
Dropped FLAG_DOCUMENT_2
Dropped FLAG_DOCUMENT_4
Dropped FLAG_DOCUMENT_7
Dropped FLAG_DOCUMENT_9
Dropped FLAG_DOCUMENT_10
Dropped FLAG_DOCUMENT_11
Dropped FLAG_DOCUMENT_12
Dropped FLAG_DOCUMENT_13
Dropped FLAG_DOCUMENT_14
Dropped FLAG_DOCUMENT_15
Dropped FLAG_DOCUMENT_16
Dropped FLAG_DOCUMENT_17
Dropped FLAG_DOCUMENT_18
Dropped FLAG_DOCUMENT_19
Dropped FLAG_DOCUMENT_20
Dropped FLAG_DOCUMENT_21


,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_3,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_8,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,...,1,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,...,1,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,297000.0,...,1,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
4,100007,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,513000.0,...,0,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0


In [62]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
cat = [np.str, np.object]
cat_features = []
for col in x_data.columns:
    if x_data[col].values.dtype in cat:
        x_data[col] = x_data[col].values.astype(np.str)
        x_data[col] = encoder.fit_transform(x_data[col].values)
        cat_features.append(col)
        print('To labels ' + col)

To labels NAME_CONTRACT_TYPE
To labels CODE_GENDER
To labels FLAG_OWN_CAR
To labels FLAG_OWN_REALTY
To labels NAME_TYPE_SUITE
To labels NAME_INCOME_TYPE
To labels NAME_EDUCATION_TYPE
To labels NAME_FAMILY_STATUS
To labels NAME_HOUSING_TYPE
To labels OCCUPATION_TYPE
To labels WEEKDAY_APPR_PROCESS_START
To labels ORGANIZATION_TYPE
To labels FONDKAPREMONT_MODE
To labels HOUSETYPE_MODE
To labels WALLSMATERIAL_MODE
To labels EMERGENCYSTATE_MODE


In [63]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.3, random_state=0)

__Задание 2. (1.5 балла)__

Обучите реализации градиентного бустинга LightGBM и Catboost на вещественных признаках без подбора параметров. 
Почему получилась заметная разница в качестве? 

В этом и последующих экспериментах необходимо измерять время обучения моделей.

In [80]:
from sklearn.metrics import auc, precision_recall_curve
def get_score(y_true, y_prob):
    precision, recall = precision_recall_curve(y_true, y_prob)[:2]
    return auc(recall, precision)

In [81]:
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from time import time

In [82]:
model = CatBoostClassifier(logging_level="Silent")
cur = time()
model.fit(x_train[numeric_features], y_train)
print('Train time %.3fs' % (time() - cur))
y_pred = model.predict_proba(x_test[numeric_features])

Train time 36.694s


In [83]:
print(get_score(y_test, y_pred[:, 1]))

0.1468489284742004


In [84]:
model = LGBMClassifier()
cur = time()
model.fit(x_train[numeric_features], y_train)
print('Train time %.3fs' % (time() - cur))
y_pred = model.predict_proba(x_test[numeric_features])

Train time 0.378s


In [85]:
print(get_score(y_test, y_pred[:, 1]))

0.1386621744470961


__Задание 3. (2 балла)__

Подберите с CV=3 оптимальные параметры алгоритмов, изменяя:

* глубину деревьев;
* количество деревьев;
* темп обучения;
* оптимизируемый функционал.

Проанализируйте соотношения глубины и количества деревьев в зависимости от алгоритма.

In [90]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
def fit_best(model, x_train, y_train, params):
    grid_search = GridSearchCV(model, param_grid=params, cv = 3, scoring=make_scorer(get_score, needs_proba=True))
    grid_search.fit(x_train, y_train)
    model.set_params(**(grid_search.best_params_))
    cur = time()
    model.fit(x_train, y_train)
    print('Train time %.3fs' % (time() - cur))
    return model

In [93]:
params = {
    "max_depth": [5, 6, 7],
    "learning_rate" : [0.01, 0.1],
    "n_estimators": [200, 500, 700]
    "eval_metric" : ['logloss', 'auc']
}
model_lgbm = fit_best(LGBMClassifier(), x_train[numeric_features], y_train, params)

/home/nikitos/.local/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Train time 0.349s


In [98]:
y_pred = model_lgbm.predict_proba(x_test[numeric_features])
print(get_score(y_test, y_pred[:, 1]))

0.16066170150237088


In [105]:
params = {
    "max_depth": [5],
    "learning_rate" : [0.01],
    "n_estimators": [200],
    "eval_metric" : ['Logloss', 'AUC']
}
model_catboost = fit_best(CatBoostClassifier(logging_level='Silent'), x_train[numeric_features], y_train, params)

Train time 4.036s


In [106]:
y_pred = model_catboost.predict_proba(x_test[numeric_features])
print(get_score(y_test, y_pred[:, 1]))

0.15301323434707179


__Задание 4. (3.5 балла)__

Добавьте категориальные признаки к вещественным следующими способами:

* как OHE признаки;
* как счетчики со сглаживанием.

При подсчете счетчиков запрещается использование циклов. 

На получившихся датасетах подберите параметры у каждого из алгоритмов. Как меняется время, необходимое для обучения модели в зависимости от способа кодирования? Сравните полученные результаты с встроенными методами обработки категориальных признаков. 

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ

__Задание 5. (1 балл)__

Реализуйте блендинг подобранных в предыдущем задании моделей и сравните качество.

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ

__Задание 6. (1.5 балла)__

В задании 3 вы подобрали гиперпараметры для LightGBM и CatBoost на вещественных признаках. Визуализируйте важности признаков, посчитанные этими алгоритмами, в виде горизонтального bar-plot (отсортируйте признаки по убыванию важности, подпишите названия признаков по оси y).

Для каждого из двух алгоритмов удалите неважные признаки (обычно по bar-plot хорошо видно порог на важность, с которого начинается хвост неважных признаков) и обучите ту же модель на получившихся данных. Сильно ли упало качество при удалении признаков, которые модель считает неважными?

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ